# Roof trought sheaf of planes

## Model of Roof:

![Caption Figura](images/mansardRoof.png)

In [ ]:
from larlib import *

parLineDis = .8

def mkpolsPerpLine(A,B):
    x1,y1,z1 = A
    x2,y2,z2 = B
    
    angle = ATAN2([ (y1-y2), (x2-x1) ])
    
    A1 = [x1+COS(angle)*parLineDis, y1+SIN(angle)*parLineDis,z1]
    B2 = [x2+COS(angle)*parLineDis, y2+SIN(angle)*parLineDis,z2]
    
    s = STRUCT(MKPOLS( [[A,B,A1,B2], [[0,1,2,3]], None ] ))
    return s

def linesPitch(A,B,rad):
    x1,y1,z1 = A
    x2,y2,z2 = B
    
    objRot = mkpolsPerpLine(A,B)
    angle = ATAN2([ (x2-x1), (y1-y2) ])
    toOrigin = T([1,2])([-A[0], -A[1]])(objRot)
    toOrigin = R([1,2])(angle)(toOrigin)
    rotation = R([2,3])(rad)(toOrigin)
    originPitch = R([1,2])(-angle)(rotation)
    
    return T([1,2])([A[0], A[1]])(originPitch)

def p(A):
    l = []
    for i in A:
        l.append(round(i, 5))
    return l

#plane passing through 3 points
def planeEquation(A,B,C):
    x1,y1,z1 = p(A)
    x2,y2,z2 = p(B)
    x3,y3,z3 = p(C)
    
    v1 = [x2 - x1, y2 - y1, z2 - z1]
    v2 = [x3 - x1, y3 - y1, z3 - z1]

    vProduct = [v1[1] * v2[2] - v1[2] * v2[1], -1 * (v1[0] * v2[2] - v1[2] * v2[0]), v1[0] * v2[1] - v1[1] * v2[0]]

    a = vProduct[0]
    b = vProduct[1]
    c = vProduct[2]
    d = - (vProduct[0] * x1 + vProduct[1] * y1 + vProduct[2] * z1)
    return [a,b,c,d]

#intersection of 3 planes
def interPlanes(p1, p2, p3):
    A = numpy.array([p1[:3], p2[:3], p3[:3]])
    B = -1*numpy.array([p1[-1], p2[-1], p3[-1]])
    return p(list(numpy.linalg.solve(A,B)))

#roof from the intersection points at the height of z
def roof(l, z):
    pol = []
    for i in range(len(l)):
        p1x,p1y = l[i-1][0],l[i-1][1]
        p2x,p2y = l[i][0],l[i][1]
        pol.append(MKPOL( [ [[p1x,p1y],[p2x,p2y]],[[1,2]],None ] ))
    return T(3)(z)(SOLIDIFY(STRUCT(pol)))

def createRoof(V,F,z,rad):
    roofPlaneEq = [0.0, -0.0, 1.0, -z]
    pitches = []
    #pitch creation
    for i in F:
        A = V[i[0]-1]
        B = V[i[1]-1]
        fsa = linesPitch(A,B,rad)
        pitches.append(fsa)
        
    iPoints = []
    #find Intersection points
    for i in range(len(pitches)):
        #find equation of each pitch
        pol1 = UKPOL(pitches[i-1])[0]
        a,b,c= p(pol1[0]), p(pol1[1]), p(pol1[2])

        pol2 = UKPOL(pitches[i])[0]
        a1,b1,c1= p(pol2[0]), p(pol2[1]), p(pol2[2])

        p1 = planeEquation(a,b,c)
        p2 = planeEquation(a1,b1,c1)

        px = interPlanes(p1,p2,roofPlaneEq)
        iPoints.append(px)
    
    
    #create Pitches from intersection Points
    intP = []
    for i in range(len(iPoints)):
        v1 = V[i-1]
        v2 = V[i]
        i2 = iPoints[i]
        i1 = iPoints[i-1]
        pitch = MKPOL([ [v1,v2,i2,i1],[[1,2,3,4]],None ])
        intP.append(pitch)
    
    return [STRUCT(intP), roof(iPoints, z)]

#test
#h of the rood
z = .5

#roof1
V = [[0,0,0],[10,0,0],[10,4,0],[4,4,0],[4,10,0],[0,10,0]]
F = [[1,2],[2,3],[3,4],[4,5],[5,6],[6,1]]

cover, roof = createRoof(V,F,z,PI/4)

cover = TEXTURE(["texture/roof.jpg", TRUE, FALSE, 1, 1, 0, 6, 6])(cover)
roof = TEXTURE(["texture/terrazzo.jpg", TRUE, FALSE, 1, 1, 0, 6, 6])(roof)
VIEW(STRUCT([cover, roof]))

#roof2
V = [[0,0,0],[4,0,0],[4,1,0],[1,1,0],[1,4,0],[0,4,0]]
F = [[1,2],[2,3],[3,4],[4,5],[5,6],[6,1]]

falde, tetto = createRoof(V,F,z,PI/4)
falde = TEXTURE(["texture/roof.jpg", TRUE, FALSE, 1, 1, 0, 6, 6])(cover)
tetto = TEXTURE(["texture/terrazzo.jpg", TRUE, FALSE, 1, 1, 0, 6, 6])(roof)
VIEW(STRUCT([cover, roof]))



Evaluating fenvs.py..
...fenvs.py imported in 0.005548 seconds


/Users/Andrea/anaconda/lib/python2.7/site-packages/larlib/larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


## Results:

![Caption Figura](images/roof1.png)

![Caption Figura](images/roof12.png)

![Caption Figura](images/roof21.png)

![Caption Figura](images/roof22.png)